## **Processador Quântico - Processador, Instruções e Execução de Eventos**

Neste notebook, iremos aprender como usar um processador quântico com instruções físicas específicas. Isso nos permite levar em consideração a duração física de um processo e incluir possíveis erros. Também começaremos a usar simulação de eventos discretos onde a operação do sistema é modelada como uma sequência discreta de eventos no tempo. Cada evento ocorre em um instante específico e altera o estado do sistema. Entre eventos, presume-se que o sistema não muda.

In [1]:
# Importanto os conteúdos necessários do pacote netsquid
import netsquid as ns
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.components.qprogram import QuantumProgram
import netsquid.components.instructions as instr
import pydynaa

<br><br>No último notebook aprendemos como criar uma memória quântica com um número predefinido de posições (num_positions) para armazenar qubits e como usar instruções para operar nos qubits. <br> <br>

Até então, as instruções não eram físicamente precisas. Eram aplicadas instantaneamente e sem erros. <br><br>

O Processador quântico nos permite mudar isso! Em um processador quântico podemos especificar um conjunto de instruções físicas com tempos de operação associados, modelos de erro e até mesmo definindo uma topologia (em quais qubits uma instrução pode ser aplicada). <br><br>

Como primeiro exemplo, especificamos as três instruções físicas a seguir:
- Inicializar um qubit leva 3 nanossegundos;
- Aplicar a porta Hadamard leva 1 nanossegundo;
- Aplicar a porta X leva 1 nanossegundo. <br> <br>

<span style="color:orange"> 
    class netsquid.components.qprocessor.PhysicalInstruction(instruction, double duration, topology = None, quantum_noise_model = None, classical_noise_model = None, bool parallel=False, apply_q_noise_after = True, **parameters)
    
</span>

In [2]:
phys_instructions1 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_H, duration=1),   
    PhysicalInstruction(instr.INSTR_X, duration=1),   
]

<br><br>Em seguida, nós inicializamos um processador quântico com uma memória quântica contendo uma posição de memória. Nós usaremos três instruções físicas especificadas acima. <br> <br>

<span style="color:orange">
    class netsquid.components.qprocessor.QuantumProcessor(name, num_positions=1, mem_noise_models=None, phys_instructions= None, mem_pos_types=None, fallback_to_nonphysical=False, properties=None, ** kwargs)

In [3]:
# Criando um processador quântico com um a posição de memória e capaz de aplicar as instruções físicas acima.
qprocl = QuantumProcessor(name="ExampleQProcl",num_positions=1, phys_instructions=phys_instructions1)

O NetSquid usa simulação de eventos discretos onde a operação do sistema é modelada como uma sequência discreta de eventos no tempo. Cada evento ocorre em um instante específico e altera o estado do sistema. Entre os eventos, presume-se que o sistema não muda. <br><br>

Podemos verificar a hora atual do sistema com ns.sim_time(). Como até o momento não agendamos eventos, o tempo não evoluiu.

In [4]:
ns.sim_time() # Checando tempo atual da simulação

0.0

A execução de uma instrução fará avançar o tempo. Para mostrar isso, vamos agendar o evento de inicialização do qubit em nosso processador. Para aplicar a instrução, precisamos executar o evento correspondente. Ns.sim_run() executa o próximo evento agendado. Por último, verificamos a hora atual, o que nos mostra que, conforme esperado, foram necessários 3 nanossegundos para inicializar o qubit. <br><br>

<span style="color: orange"> class netsquid.components.qprocessor.QuantumProcessor.execute_instruction(self, instruction, qubit_mapping=None, str output_key='instr',bool physical=True, bool check_qubit_mapping=True, **parameters) </span>

In [5]:
qprocl.execute_instruction(instr.INSTR_INIT, [0]) # Agenda a inicialização do qubit no slot 0
ns.sim_run() # Roda o próximo evento agendado
ns.sim_time() # checa o tempo atual em ns

3.0

Em seguida, nós agendamos a aplicação da porta H. Executamos esse evento e novamente verificamos a evolução temporal.

In [6]:
qprocl.execute_instruction(instr.INSTR_H, [0]) # Agendando a aplicação de H no slot 0
ns.sim_run() # Roda o próximo evento agendado
ns.sim_time() # checa o tempo atual em ns

4.0

Como antes, podemos espiar o estado do qubit,

In [10]:
qprocl_s0, = qprocl.peek(positions=[0]) # Espiando o qubit na posição 0.
print("qprocl_s0 as ket", qprocl_s0.qstate.qrepr) # Mostra o estado do qubit na posição 0.

qprocl_s0 as ket KetRepr(num_qubits=1,
ket=
[[0.70710678+0.j]
 [0.70710678+0.j]])


<span style = "color: red"> **Tentar executar uma instrução que não pode corresponder a uma instrução física causará um erro como pode ser visto abaixo** </span>

In [11]:
qprocl.execute_instruction(instr.INSTR_Z, [0]) # Agendando a aplicação uma porta Z no slot 0
ns.sim_run() # Executa o próximo evento agendado.

MissingInstructionError: Missing physical instruction for Instruction: z_gate

<span style="color: green">**Para corrigir este erro, você precisari adicionar a porta Z em suas instruções físicas.**  $$$$Ao invés de:
</span>

In [12]:
phys_instructions1 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3), # Inicializando um qubit
    PhysicalInstruction(instr.INSTR_H, duration=1),    # Porta H
    PhysicalInstruction(instr.INSTR_X, duration=1),    # Porta X
]

<span style="color: green">você precisa escrever:
</span>

In [13]:
phys_instructions1 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3), # Inicializando um qubit
    PhysicalInstruction(instr.INSTR_H, duration=1),    # Porta H
    PhysicalInstruction(instr.INSTR_X, duration=1),    # Porta X
    PhysicalInstruction(instr.INSTR_Z, duration=1),    # Porta Z
]

## **Sugestões de Prática**

- Especifique um conjunto de instrunções físicas que permite inicializar o qubit (duração 5 ns), aplique as portas H, X, Y e Z (duração de 2 ns cada); 
- Inicialize um novo processador quântico que chame esse conjunto de instruções físicas;
- Aplique a porta H no qubit. Rode esse evento e cheque o tempo;
- Aplique a porta Y no qubit. Rode esse evento e cheque o tempo;
- Aplique a porta Z no qubit. Rode esse evento e cheque o tempo;
- Espie o estado do qubit.